In [2]:
import pdfplumber
import os

# PDF文件路径
pdf_path = "/Users/tangluoxi/Desktop/Education/Verbal Workout GRE 6ed ( PDFDrive ).pdf"

# 输出TXT文件的路径（与PDF同文件夹）
output_path = os.path.join(os.path.dirname(pdf_path), "Extracted_Text.txt")

# 打开PDF文件
with pdfplumber.open(pdf_path) as pdf:
    all_text = ''
    for page in pdf.pages:
        # 提取当前页的文本
        text = page.extract_text()
        if text:
            all_text += text + '\n'

# 将提取的文本保存到TXT文件
with open(output_path, "w") as text_file:
    text_file.write(all_text)

print(f"Text extracted and saved to {output_path}")


Text extracted and saved to /Users/tangluoxi/Desktop/Education/Extracted_Text.txt


In [1]:
pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 596.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import re
from collections import OrderedDict

def extract_drills(document):
    # 分割文档为各个练习部分
    parts = re.split(r'(Practice Drill #\d+)', document)
    drills = OrderedDict()
    
    for i in range(1, len(parts), 2):
        header = parts[i].strip()
        content = parts[i+1].strip()
        # 提取编号
        match = re.match(r'Practice Drill #(\d+)', header)
        if not match:
            continue
        number = match.group(1)
        # 合并相同编号的内容，只保留第一个标题
        if number in drills:
            drills[number] += '\n' + content  # 仅追加内容，不重复标题
        else:
            drills[number] = header + '\n' + content
    
    # 按出现的顺序打印合并后的练习
    for idx, (number, drill) in enumerate(drills.items(), 1):
        print(f"Practice Drill {idx}:\n{drill}\n{'='*50}\n")

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

file_path = '/Users/tangluoxi/Desktop/Education/Extracted_Text.txt'
document_content = read_file(file_path)
extract_drills(document_content)

Practice Drill 1:
Practice Drill #1
Directions: Each sentence below has one, two, or three blanks, each blank
indicating that something has been omitted. Beneath the sentence are five
lettered words or sets of words. Choose the word or set of words for each blank
that best fits the meaning of the sentence as a whole. The answers are on this
page.
1. The couple thought that the values represented in traditional
services were _________, so they decided to write their own, more
modern, wedding vows.
bigoted
archaic
dogmatic
ineloquent
somber
2. An experienced film critic is one who not only calls attention to the
(i) ________ of a particular feature, but also puts forth legitimate (ii)
________ that, if employed, would create a more satisfying product.
Blank (i) Blank (ii)
fascinations recommendations
origins statistics
shortcomings controversies
3. Louis was so painfully shy that his friends had to cajole him not to
_____ even the smallest social gathering.
confront
confront
subdue
flout

In [19]:
import re
import json
import os

class QuestionParser:
    @staticmethod
    def detect_question_type(text):
        """增强的题型检测逻辑"""
        text = text.replace("\n", " ")
        if re.search(r"Blank \(i\)", text):
            return "fill_in_blanks"
        elif re.search(r"Choose the word.*best fits", text):
            return "single_choice"
        elif re.search(r"lettered words.*\d+ blanks", text):
            return "multi_blank"
        return "unknown"

    @staticmethod
    def parse_fill_in_blanks(text):
        """改进的填空题解析（支持2-3空）"""
        result = {
            "type": "fill_in_blanks",
            "blanks": [],
            "correct_answer": [],
            "analysis": ""
        }
        
        # 提取题干
        if (match := re.search(r"(\d+\.\s+.*?)\nBlank", text, re.DOTALL)):
            result["question"] = re.sub(r"\s+", " ", match.group(1)).strip()

        # 提取所有空白选项
        blank_sections = re.finditer(r"Blank \((i+)\)(.*?)(?=Blank|The correct)", text, re.DOTALL)
        for match in blank_sections:
            blank_id = match.group(1)
            options = [x.strip() for x in match.group(2).split("\n") if x.strip()]
            result["blanks"].append({
                "blank_id": blank_id,
                "options": options
            })

        # 提取答案
        if (match := re.search(r"The correct answer is \((.*?)\)(.*?)(?=\n\d+\.|\Z)", text, re.DOTALL)):
            answers = re.findall(r"[A-E]", match.group(1))
            result["correct_answer"] = answers
            result["analysis"] = re.sub(r"\s+", " ", match.group(2)).strip()
        
        return result

    @staticmethod
    def parse_single_choice(text):
        """改进的单选题解析"""
        result = {
            "type": "single_choice",
            "options": [],
            "correct_answer": [],
            "analysis": ""
        }
        
        # 提取题干和选项
        if (match := re.search(r"(\d+\.\s+.*?)\n(([A-E]\.\s+.*\n)+)", text, re.DOTALL)):
            result["question"] = re.sub(r"\s+", " ", match.group(1)).strip()
            result["options"] = [
                re.sub(r"^[A-E]\.\s*", "", x).strip()
                for x in match.group(2).split("\n")
                if x.strip()
            ]

        # 提取答案
        if (match := re.search(r"The correct answer is (\([A-E]\))", text)):
            result["correct_answer"] = re.findall(r"[A-E]", match.group(1))
            result["analysis"] = re.sub(r"\s+", " ", text.split(match.group(1))[1]).strip()
        
        return result

    @staticmethod
    def parse_multi_blank(text):
        """多空题解析（特殊题型）"""
        # 实现逻辑与fill_in_blanks类似，根据实际情况调整
        return QuestionParser.parse_fill_in_blanks(text)

    @staticmethod
    def parse_unknown(text):
        """增强的未知题型处理"""
        return {
            "type": "unknown",
            "raw_content": re.sub(r"\s+", " ", text).strip(),
            "warning": "Unrecognized question format"
        }

def process_document(document):
    """改进的文档处理流程"""
    # 分割练习章节
    drills = re.split(r'(Practice Drill #\d+)', document)
    output = []
    
    for i in range(1, len(drills), 2):
        drill_header = drills[i].strip()
        content = drills[i+1].strip()
        
        # 提取章节编号
        if not (drill_num := re.search(r"\d+", drill_header)):
            continue
        
        # 改进的题目分割
        questions = re.split(r'\n(?=(\d+)\.)', content)
        drill_data = {
            "drill_number": int(drill_num.group()),
            "questions": []
        }
        
        for q_idx in range(1, len(questions), 2):
            q_num = questions[q_idx].strip()
            q_text = questions[q_idx+1].strip()
            
            # 解析题型
            q_type = QuestionParser.detect_question_type(q_text)
            parser = getattr(QuestionParser, f"parse_{q_type}", QuestionParser.parse_unknown)
            parsed = parser(q_text)
            parsed["question_number"] = int(q_num)
            
            drill_data["questions"].append(parsed)
        
        output.append(drill_data)
    
    return output

def save_to_json(data, path):
    """保存JSON文件"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"文件已保存至：{path}")

# 使用示例
if __name__ == "__main__":
    input_file = "/Users/tangluoxi/Desktop/Education/Extracted_Text.txt"
    output_file = "/Users/tangluoxi/Desktop/Education/Processed_Drills.json"
    
    with open(input_file, 'r', encoding='utf-8') as f:
        document = f.read()
    
    result = process_document(document)
    save_to_json(result, output_file)

文件已保存至：/Users/tangluoxi/Desktop/Education/Processed_Drills.json


In [20]:
import re
import os
from collections import OrderedDict

def extract_drills(document, output_dir):
    """
    提取练习题并保存到指定目录
    :param document: 文本内容
    :param output_dir: 输出目录路径
    """
    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)
    
    # 分割文档为各个练习部分
    parts = re.split(r'(Practice Drill #\d+)', document)
    drills = OrderedDict()
    
    # 处理每个练习模块
    for i in range(1, len(parts), 2):
        header = parts[i].strip()
        content = parts[i+1].strip()
        
        # 提取编号
        if match := re.match(r'Practice Drill #(\d+)', header):
            number = match.group(1)
            # 合并相同编号的内容
            if number in drills:
                drills[number] += '\n' + content
            else:
                drills[number] = f"{header}\n{content}"

    # 保存到文件
    output_path = os.path.join(output_dir, "Extracted_Drills.txt")
    with open(output_path, 'w', encoding='utf-8') as f:
        for idx, (number, drill) in enumerate(drills.items(), 1):
            f.write(f"Practice Drill {idx}:\n{drill}\n{'='*50}\n\n")
    
    print(f"成功提取并保存到：{output_path}")

def read_file(file_path):
    """读取文件内容"""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 使用路径
input_file = '/Users/tangluoxi/Desktop/Education/Extracted_Text.txt'
output_dir = '/Users/tangluoxi/Desktop/Education'

# 执行流程
document_content = read_file(input_file)
extract_drills(document_content, output_dir)

成功提取并保存到：/Users/tangluoxi/Desktop/Education/Extracted_Drills.txt


In [39]:
import re
import json

# -------------------------------
# 配置输入和输出文件路径
# -------------------------------
input_file = "/Users/tangluoxi/Desktop/Education/Extracted_Drills.txt"
output_file = "/Users/tangluoxi/Desktop/Education/Structured_Questions.json"

# -------------------------------
# 1. 读取输入文件内容
# -------------------------------
with open(input_file, 'r', encoding='utf-8') as f:
    content = f.read()

# -------------------------------
# 2. 定位题目部分
#
#    假设题目部分从第一行以 "1." 开头处开始（跳过题目前的说明文字）
# -------------------------------
match_first = re.search(r'^\s*1\.\s', content, flags=re.MULTILINE)
if match_first:
    question_content = content[match_first.start():]
else:
    question_content = content

# -------------------------------
# 3. 利用正则表达式提取每个题目块
#
#    每个题目块格式为：数字. 后跟若干文本，直到下一个题号出现或文件结束
# -------------------------------
pattern = re.compile(r'(?P<number>\d+)\.\s*(?P<text>.*?)(?=\n\d+\.|$)', re.DOTALL)
matches = list(pattern.finditer(question_content))

questions = []  # 存储解析后的题目列表

for m in matches:
    q_num = int(m.group("number"))
    block_text = m.group("text").strip()
    if not block_text:
        continue

    # 按行拆分并去除空行
    lines = [line.strip() for line in block_text.splitlines() if line.strip() != ""]

    # 如果没有有效行，则跳过
    if not lines:
        continue

    # -------------------------------
    # 4. 根据内容判断题型
    #
    #    如果任一行中包含 "Blank"（大小写不敏感），则视为填空题；否则视为单项选择题
    # -------------------------------
    if any("blank" in line.lower() for line in lines):
        question_type = "fill_in_blanks"
        # 查找包含 "Blank" 的行作为选项表头，header_index 为该行索引
        header_index = None
        for i, line in enumerate(lines):
            if "blank" in line.lower():
                header_index = i
                break
        if header_index is None:
            header_index = len(lines)
        # 题干为 header_index 之前的所有行合并而成
        question_statement = " ".join(lines[:header_index])
        # 选项表格部分：header 行之后的所有行
        table_rows = lines[header_index+1:]
        # 将每一行按空白字符拆分为列表
        table = [row.split() for row in table_rows if row]

        # 从题干中提取所有空的标记（如 (i), (ii), ...），不区分大小写
        blank_ids = re.findall(r'\(([ivx]+)\)', question_statement, flags=re.IGNORECASE)
        blanks = []
        for col_index, blank_id in enumerate(blank_ids):
            col_options = []
            for row in table:
                if len(row) > col_index:
                    col_options.append(row[col_index])
            blanks.append({
                "blank_id": blank_id,
                "options": col_options
            })
        q_dict = {
            "type": question_type,
            "question_number": q_num,
            "question": question_statement,
            "blanks": blanks,
            "correct_answer": [],
            "analysis": ""
        }
    else:
        # 处理单项选择题
        question_type = "single_choice"
        # 这里假设单项选择题最后5行为选项，前面的行合并为题干
        if len(lines) > 5:
            options = lines[-5:]
            question_statement = " ".join(lines[:-5])
        else:
            # 如果行数不足，则第一行为题干，其余行为选项
            question_statement = lines[0]
            options = lines[1:]
        q_dict = {
            "type": question_type,
            "question_number": q_num,
            "question": question_statement,
            "options": options,
            "correct_answer": [],
            "analysis": ""
        }
    questions.append(q_dict)

# -------------------------------
# 5. 构造最终 JSON 数据结构
# -------------------------------
final_data = {
    "drill_number": 1,
    "questions": questions
}

# -------------------------------
# 6. 写入 JSON 文件
# -------------------------------
with open(output_file, 'w', encoding='utf-8') as f_out:
    json.dump(final_data, f_out, indent=2, ensure_ascii=False)

print("Structured JSON has been written to", output_file)


Structured JSON has been written to /Users/tangluoxi/Desktop/Education/Structured_Questions.json


In [1]:
import re
import json

# ----------------------------------------
# 1. 配置输入和输出文件路径
# ----------------------------------------
input_file = "/Users/tangluoxi/Desktop/Education/Extracted_Drills.txt"
output_file = "/Users/tangluoxi/Desktop/Education/Structured_Questions.json"

# ----------------------------------------
# 2. 读取整个输入文件内容
# ----------------------------------------
with open(input_file, 'r', encoding='utf-8') as f:
    content = f.read()

# ----------------------------------------
# 3. 利用正则表达式提取所有“块”
#    模式说明：
#      (?P<number>\d+)\.\s*     匹配题号（例如 "1."），捕获数字部分
#      (?P<text>.*?)(?=\n\d+\.|\Z) 捕获该块内的文本，直到遇到下一块或结束
#    注意：答案部分也符合该模式
# ----------------------------------------
pattern = re.compile(r'(?P<number>\d+)\.\s*(?P<text>.*?)(?=\n\d+\.|\Z)', re.DOTALL)
matches = list(pattern.finditer(content))

# ----------------------------------------
# 4. 根据题号是否递增来区分题目块和答案块
#    题目部分中题号应是依次递增的；一旦遇到题号小于或等于前面的，
#    则认为从此开始为答案部分。
# ----------------------------------------
questions_matches = []  # 存储题目部分的 Match 对象
answer_matches = []     # 存储答案部分的 Match 对象

max_q = 0
for m in matches:
    q_num = int(m.group("number"))
    text = m.group("text").strip()
    if not text:
        continue
    if q_num > max_q:
        # 题号递增，视为题目块
        questions_matches.append(m)
        max_q = q_num
    else:
        # 题号出现回绕，视为答案块
        answer_matches.append(m)

# ----------------------------------------
# 5. 处理答案块，建立映射：题号 -> {correct_answer, analysis}
#
#    对于答案块，其文本中会包含正确答案的标识，
#    我们采用正则提取所有形如 (B) 的字母，作为正确答案。
#    同时将整个答案块内容作为解析文本保存。
# ----------------------------------------
answer_map = {}
for m in answer_matches:
    q_num = int(m.group("number"))
    text = m.group("text").strip()
    # 提取所有大写字母，形如 (B) 或 (C)
    letters = re.findall(r'\(([A-Z])\)', text)
    answer_map[q_num] = {"correct_answer": letters, "analysis": text}

# ----------------------------------------
# 6. 根据题目块构造题目的 JSON 数据结构
#
#    对于每个题目块：
#      - 按行拆分（去除空行）；
#      - 判断题型：若任一行包含 "blank"（不区分大小写），视为填空题，
#        否则为单项选择题。
#
#    单项选择题假定最后若干行为选项（这里简单假定若行数大于5，则最后5行为选项）；
#    填空题中，题干中会出现空的标记 (i)、(ii) 等，其选项通常在包含“Blank”字样的行之后，
#    此处按表格拆分处理（每行按空白拆分，按列对应各空）。
# ----------------------------------------
questions = []

for m in questions_matches:
    q_num = int(m.group("number"))
    block_text = m.group("text").strip()
    # 按行拆分，并去除两端空白
    lines = [line.strip() for line in block_text.splitlines() if line.strip()]
    if not lines:
        continue

    # 判断题型：如果任一行中含有 "blank"（不区分大小写），则视为填空题
    if any("blank" in line.lower() for line in lines):
        q_type = "fill_in_blanks"
        # 查找包含 "blank" 的行作为选项表头行，header_index 即为该行索引
        header_index = None
        for i, line in enumerate(lines):
            if "blank" in line.lower():
                header_index = i
                break
        if header_index is None:
            header_index = len(lines)
        # 题干部分为 header_index 之前的所有行合并为一句话
        question_statement = " ".join(lines[:header_index])
        # 表格部分：header_index 后的所有行；每行按空白拆分成列表
        table_rows = lines[header_index+1:]
        table = [row.split() for row in table_rows if row]
        # 从题干中提取所有空的标记，例如 (i)、(ii) 等（不区分大小写）
        blank_ids = re.findall(r'\(([ivx]+)\)', question_statement, flags=re.IGNORECASE)
        blanks = []
        for col_index, blank_id in enumerate(blank_ids):
            col_options = []
            for row in table:
                if len(row) > col_index:
                    col_options.append(row[col_index])
            blanks.append({
                "blank_id": blank_id,
                "options": col_options
            })
        q_dict = {
            "type": q_type,
            "question_number": q_num,
            "question": question_statement,
            "blanks": blanks,
            "correct_answer": [],
            "analysis": ""
        }
    else:
        # 单项选择题处理
        q_type = "single_choice"
        # 简单假定：若行数大于5，则最后5行为选项，其余合并为题干；
        # 否则，第一行为题干，剩余行为选项
        if len(lines) > 5:
            options = lines[-5:]
            question_statement = " ".join(lines[:-5])
        else:
            question_statement = lines[0]
            options = lines[1:]
        q_dict = {
            "type": q_type,
            "question_number": q_num,
            "question": question_statement,
            "options": options,
            "correct_answer": [],
            "analysis": ""
        }
    # 如果该题在答案映射中，则填充答案和解析
    if q_num in answer_map:
        q_dict["correct_answer"] = answer_map[q_num]["correct_answer"]
        q_dict["analysis"] = answer_map[q_num]["analysis"]
    questions.append(q_dict)

# ----------------------------------------
# 7. 构造最终 JSON 数据结构，并写入输出文件
# ----------------------------------------
final_data = {
    "drill_number": 1,
    "questions": questions
}

with open(output_file, 'w', encoding='utf-8') as f_out:
    json.dump(final_data, f_out, indent=2, ensure_ascii=False)

print("Structured JSON has been written to", output_file)


Structured JSON has been written to /Users/tangluoxi/Desktop/Education/Structured_Questions.json
